In [11]:
import os
from os.path import expanduser
import socket
import sys


# BELOW IS SYSTEM/PATH CONFIGURATION #
my_home = expanduser("~")
running_system = socket.gethostname()
# Configure machine's names here #
systemHLRN = 'blogin1'
systemLOCAL = 'Laurents-MacBook-Pro.local'
my_home = expanduser("~")
# Define local pass #
if running_system == systemLOCAL:
    print "computer = ", running_system
    wd_path = os.path.join(my_home,"Documents","projects", "awi-models")
elif running_system[0:1] == "b" or running_system == systemHLRN:
    print "computer = ", running_system
    wd_path = os.path.join(my_home,"awi-models")
else:
    print 'please configure your local machine : type socket.gethostname()'
print "working directory set to", wd_path
os.chdir(wd_path)
# IMPORT PY-FESOM MODULES
sys.path.append("codes/modules")
sys.path.append("codes/pyfesom-master/") # add pyfesom to search path
#########################################

computer =  blogin1
working directory set to /home/hbkoziel/awi-models


In [12]:
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap
from pylab import *
from load_mesh_data_new import *
import numpy as np
import datetime as dt
from PiecewiseNorm import PiecewiseNorm
from netCDF4 import Dataset
#import colormaps as cmaps
from matplotlib.colors import ListedColormap
import pyfesom as pf

In [13]:
# PLOT CONFIGURATION #

# load colormaps #
cmap = plt.get_cmap('RdBu_r')
# figure export 
export_plot = True
# figure export definition
dpicnt=150
# choose simulation
runid='Arc12'
# choose date
first_year = 1980
last_year  = 2015
years      = np.arange(first_year,last_year+1,1)
months =np.linspace(0,11,12).astype(int)
# choose depth
get3d = False
# load the given biological tracer #
phyc_id, phychl_id, diac_id, diachl_id, par_id, temp_id, depth	='tr05', 'tr06', 'tr14', 'tr15', 'PAR3D_mean', 'temp', 0
phyn_id, dian_id, diasi_id, din_id, dsi_id, dfe_id, ice_id	='tr04', 'tr13', 'tr16', 'tr01', 'tr18', 'tr19', 'area'
# export CSV
export_csv = True
# model parameters
alfa_phy, alfa_dia, Pcm, C2K, rTref, Ae = 0.19, 0.26, 3.0, 273.15, 1/288.15, 4500.0
NMinSlope, SiMinSlope, NCmin, SiCmin  = 50, 1000, 0.04, 0.04

########################

# AUTOMATIC DEFINITION OF PATHS
resultpath = '/scratch/usr/hbkvsk12/hlrn3_work2/results/'+runid+'/Oldfiles/'
savepath    = wd_path+'/figures/'+runid+'/'
meshpath    = wd_path+'/data/mesh/meshArc4.5/'
outputpath = '/scratch/usr/hbkoziel/Arc12/CSV/'

# Create figure directory if it does not exist
if os.path.exists(savepath) == False:
    try:
        os.mkdir(savepath)
        print ("Creation of the directory %s successfull" % savepath)
    except OSError:
        print ("Creation of the directory %s failed" % savepath)
else:
    print ("directory %s already existing" % savepath)

directory /home/hbkoziel/awi-models/figures/Arc12/ already existing


In [14]:
mesh = fesom_mesh(meshpath, get3d = get3d)

Load 2d part of the grid in 3 second(s)


In [23]:
limphy          = np.zeros((len(years),len(mesh.x2)))
limdia          = np.zeros((len(years),len(mesh.x2)))

for mo in range(0,11):
    print mo
    if mo ==0: 
        dayind = (range(0,15))
        month='JAN'
    if mo ==1: 
        dayind = (range(15,30))
        month='FEV'
    if mo ==2: 
        dayind = (range(30,45))
        month='MAR'
    if mo ==3: 
        dayind = (range(45,60))
        month='APR'
    elif mo ==4: 
        dayind = (range(60,75))
        month='MAY'
    elif mo ==5: 
        dayind = (range(75,90))
        month='JUN'
    elif mo ==6: 
        dayind = (range(90,105))
        month='JUL'
    elif mo ==7:
        dayind = (range(105,120))
        month='AUG'
    elif mo ==8: 
        dayind = (range(120,135))
        month='SEP'
    elif mo ==9: 
        dayind = (range(135,150))
        month='OCT'
    elif mo ==10: 
        dayind = (range(150,165))
        month='NOV'
    elif mo ==11: 
        dayind = (range(165,174))
        month='DEC'
        
    for ye in range(0,len(years)):
     print years[ye]
    
     ncfile  = resultpath+runid+'.'+str(years[ye])+'.oce.mean.nc'
     f0       = Dataset(ncfile, 'r')  
    
     ncfile  = resultpath+runid+'.'+str(years[ye])+'.bio.mean.nc'
     f1       = Dataset(ncfile, 'r')   
        
     if years[ye] < 2000:
        DIN       = f0.variables[din_id][mo,0:mesh.n2d]
        DSi       = f0.variables[dsi_id][mo,0:mesh.n2d]
        DFe       = f0.variables[dfe_id][mo,0:mesh.n2d]
        phyc      = f0.variables[phyc_id][mo,0:mesh.n2d]
        phyn      = f0.variables[phyn_id][mo,0:mesh.n2d]
        phychl    = f0.variables[phychl_id][mo,0:mesh.n2d]
        diac      = f0.variables[diac_id][mo,0:mesh.n2d]
        dian      = f0.variables[dian_id][mo,0:mesh.n2d]
        diasi     = f0.variables[diasi_id][mo,0:mesh.n2d]
        diachl    = f0.variables[diachl_id][mo,0:mesh.n2d]
        T         = f0.variables[temp_id][mo,0:mesh.n2d]
        par         = f1.variables[par_id][mo,0:mesh.n2d]
      
     else:
        
        DIN       = f0.variables[din_id][dayind,0:mesh.n2d].mean(axis=0)
        DSi       = f0.variables[dsi_id][dayind,0:mesh.n2d].mean(axis=0)
        DFe       = f0.variables[dfe_id][dayind,0:mesh.n2d].mean(axis=0)
        phyc      = f0.variables[phyc_id][dayind,0:mesh.n2d].mean(axis=0)
        phyn      = f0.variables[phyn_id][dayind,0:mesh.n2d].mean(axis=0)
        phychl    = f0.variables[phychl_id][dayind,0:mesh.n2d].mean(axis=0)
        diac      = f0.variables[diac_id][dayind,0:mesh.n2d].mean(axis=0)
        dian      = f0.variables[dian_id][dayind,0:mesh.n2d].mean(axis=0)
        diasi     = f0.variables[diasi_id][dayind,0:mesh.n2d].mean(axis=0)
        diachl    = f0.variables[diachl_id][dayind,0:mesh.n2d].mean(axis=0)
        T         = f0.variables[temp_id][dayind,0:mesh.n2d].mean(axis=0)
        par       = f1.variables[par_id][dayind,0:mesh.n2d].mean(axis=0)

     Nlimphy = DIN/(DIN+0.55)
     Nlimdia = DIN/(DIN+1.0)
     Silim   = DSi/(DSi+4.0)
     Felimphy= DFe/(DFe+0.02)
     Felimdia= DFe/(DFe+0.12)
    
     # Quotas
     phychlQ = phychl/phyc
     phynQ   = phyn/phyc
     diachlQ = diachl/diac
     dianQ   = dian/diac
     diasiQ  = diasi/diac
     
     # Temperature dependece
     T = 1./(T + C2K)
     T = np.exp( -Ae * (T - rTref));
    
     # Nutrient growth limitation
     Phy_dq              = NCmin - phyn
     Phy_recom_limiter   = 1.0 - np.exp( -NMinSlope*( abs(Phy_dq)-Phy_dq )**2)

     Dia_dq              = NCmin - dian
     Dia_recom_limiterN  = 1.0 - np.exp( -NMinSlope*( abs(Dia_dq)-Dia_dq )**2)
     Dia_dq              = SiCmin - diasiQ;
     Dia_recom_limiterSi = 1.0 - np.exp( -SiMinSlope*( abs(Dia_dq)-Dia_dq )**2)
        
     # Most limiting factor 
     Fephy       = DFe/(DFe+0.02)
     Phy_qlimFac = np.column_stack((Phy_recom_limiter,Fephy))
     Phy_qlimFac = Phy_qlimFac.min(axis=1)
        
     Fedia       = DFe/(DFe+0.12)
     Dia_qlimFac = np.column_stack((Dia_recom_limiterN,Dia_recom_limiterSi,Fedia))
     Dia_qlimFac = Dia_qlimFac.min(axis=1)
    
     # pmax
     Pmax_phy = Pcm * Phy_qlimFac * T
     Pmax_dia = Pcm * Dia_qlimFac * T

     Phy_Light_limiter = 1 - np.exp((-alfa_phy * phychlQ * par)/Pmax_phy)
     Dia_Light_limiter = 1 - np.exp((-alfa_dia * diachlQ * par)/Pmax_dia)
    
     # Most limiting factors found for nano
     lim=np.column_stack((Felimphy,Nlimphy,Phy_Light_limiter))
     limphy[ye,:]=lim.argmin(axis=1)
     # Most limiting factors found for dia
     lim=np.column_stack((Felimdia,Nlimdia,Silim,Dia_Light_limiter))
     limdia[ye,:]=lim.argmin(axis=1)

    
    if export_csv == True:
        np.savetxt(outputpath+'LIMphy_'+month+'_'+str(first_year)+'_'+str(last_year)+'_timeseries.csv', limphy, delimiter=";")
        np.savetxt(outputpath+'LIMdia_'+month+'_'+str(first_year)+'_'+str(last_year)+'_timeseries.csv', limdia, delimiter=";")
        print 'exporting done ...'

0
1980


/home/hbkoziel/.conda/envs/pyfesom-docs/lib/python2.7/site-packages/ipykernel_launcher.py:120: RuntimeWarning: divide by zero encountered in divide
/home/hbkoziel/.conda/envs/pyfesom-docs/lib/python2.7/site-packages/ipykernel_launcher.py:121: RuntimeWarning: divide by zero encountered in divide


1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
exporting done ...
1
1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
exporting done ...
2
1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
exporting done ...
3
1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
exporting done ...
4
1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
exporting done ...
5
